In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
beijing = pd.read_csv('Cities/Beijing_labeled.csv')
chengdu = pd.read_csv('Cities/Chengdu_labeled.csv')
guangzhou = pd.read_csv('Cities/Guangzhou_labeled.csv')
shanghai = pd.read_csv('Cities/Shanghai_labeled.csv')
shenyang = pd.read_csv('Cities/Shenyang_labeled.csv')


print(beijing.shape, chengdu.shape, guangzhou.shape, shanghai.shape, shenyang.shape)

beijing.columns


(2071, 11) (1110, 11) (1352, 11) (1351, 11) (824, 11)


Index(['season', 'DEWP', 'HUMI', 'PRES', 'TEMP', 'Iws', 'precipitation',
       'cbwd_NE', 'cbwd_NW', 'cbwd_SE', 'PM_HIGH'],
      dtype='object')

In [17]:
beijing.head()

,season,DEWP,HUMI,PRES,TEMP,Iws,precipitation,cbwd_NE,cbwd_NW,cbwd_SE,PM_HIGH
0,4,-8.0,79.0,1026.0,-5.0,23.69,0.0,0,0,1,1.0
1,4,-11.0,85.0,1021.0,-9.0,105.93,1.1,0,0,1,0.0
2,4,-21.0,43.0,1030.0,-11.0,117.55,0.0,0,1,0,0.0
3,4,-25.0,33.0,1034.0,-12.0,39.35,0.0,1,0,0,0.0
4,4,-24.0,30.0,1034.0,-10.0,59.00,0.0,1,0,0,0.0


(x - mu)/sigma

In [179]:
beijing_np = beijing.to_numpy()

In [180]:
beijing_np

array([[  4.,  -8.,  79., ...,   0.,   1.,   1.],
       [  4., -11.,  85., ...,   0.,   1.,   0.],
       [  4., -21.,  43., ...,   1.,   0.,   0.],
       ...,
       [  4.,  -8.,  50., ...,   1.,   0.,   1.],
       [  4., -11.,  28., ...,   1.,   0.,   0.],
       [  4., -10.,  37., ...,   0.,   0.,   0.]], shape=(2071, 11))

In [178]:
a

array([27.26340602, 16.62792636, 24.721978  , 25.44849879, 14.90795803,
       11.12031886, 26.87770809, 19.16532489, 17.25422165, 10.01584361])

In [181]:
mean = np.mean(beijing_np, axis=0)

In [182]:
mean

array([2.48575567e+00, 1.20183486e+00, 4.11211975e+01, 1.01671173e+03,
       1.62206663e+01, 2.14990343e+01, 4.65475616e-02, 1.13954611e-01,
       3.06132303e-01, 3.15789474e-01, 3.13375181e-01])

In [183]:
std = np.std(beijing_np, axis=0)

In [184]:
std

array([ 1.12332923, 14.61436745, 22.82766223, 10.25316677, 11.68531861,
       47.6189516 ,  0.49582166,  0.31775613,  0.46088536,  0.46482952,
        0.46386547])

In [185]:
a = (beijing_np - mean)/std

In [186]:
a[0]

array([ 1.3479969 , -0.62964305,  1.65933779,  0.90589247, -1.81601093,
        0.04601037, -0.09387965, -0.35862286, -0.66422657,  1.47196014,
        1.48022403])

In [190]:
a[0]

array([ 1.3479969 , -0.62964305,  1.65933779,  0.90589247, -1.81601093,
        0.04601037, -0.09387965, -0.35862286, -0.66422657,  1.47196014,
        1.48022403])

In [188]:
np.std(a, axis = 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [387]:
import numpy as np

class PM_HIGH_PREDICTOR:
    def __init__(self, K, tolerance=1e-5):
        self.mean = None
        self.std = None
        self.K = K
        self.tolerance=tolerance
        self.centroids = None
    
    def normalize(self, X, training = False):
        """
        Inputs:
        - X (numpy array): Array containing the input data.
        """
        if training:
            self.mean = np.mean(X, axis = 0)        # Mean over the rows of the input array
            self.std = np.std(X, axis = 0)          # STD over the rows of the input array

        return (X - self.mean)/self.std
        
    def fit(self, X_train, verbose=False):
        
        """
        Inputs:
        - X (np.array): training data
        - verbose: verbosity parameter (set to True to see distance and # of iterations)
        Outputs:
        - None
        """
        
        if len(X_train.shape) == 1:
            raise Exception('You need at least two vectors to train K-Means')
            
        ### INPUT DATA INITIALIZATION ###
        X = self.normalize(X_train, training=True)                                                           
        
        ### RANDOM CENTROID INITIALIZATION ###
        self.centroids = np.random.normal(loc=0, scale= 1, size = (self.K, X.shape[1]))     
        
        centroid_distance = 100
        n_iterations = 0
        
        
        #### ITERATIVE STEP ####
        while centroid_distance > self.tolerance:
            
            # Computing distance between each datapoint and each centroid (returns array of shape n_datapoints x n_centroids)
            point_centroid_distances = np.linalg.norm(X[:, np.newaxis, :] - self.centroids[np.newaxis, :, :], axis = 2)
            
            # Assigning each datapoint to closest centroid (returns array of shape n_datapoints x 1, with entries in [0, K])
            centroid_assignment = np.argmin(point_centroid_distances, axis = 1)

            
            new_centroids = []
            for i in range(self.K):
                if np.bincount(centroid_assignment)[i]:
                    
                    # Computing new centroids by doing mean of assigned datapoints
                    new_centroids.append(np.mean(X[centroid_assignment == i], axis = 0))
                
                else:
                    new_centroids.append(self.centroids[i])
            
            new_centroids = np.array(new_centroids)
            centroid_distance = np.linalg.norm(self.centroids - new_centroids)
            self.centroids = new_centroids
            n_iterations += 1
            if verbose:
                print(f'ITERATION {n_iterations}. DISTANCE: {centroid_distance}')
    
    def predict(self, X_test):
        if self.centroids is None:
            raise Exception('The model has not been trained')
        
        if len(X_test.shape) == 1:
            X_test = X_test[np.newaxis, :]
            
        if X_test.shape[1] != self.centroids.shape[1]:
            raise Exception(f'Doing prediction with {X_test.shape[1]} features, but trained on {self.centroids.shape[1]}')
        
        X = self.normalize(X_test, training=False)
        
        point_centroid_distances = np.linalg.norm(X[:, np.newaxis, :] - self.centroids[np.newaxis, :, :], axis = 2)
        return np.argmin(point_centroid_distances, axis = 1)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        n_labels = max(self.centroids.shape[0], int(np.max(y)))
        confusion_matrix = np.zeros((n_labels, n_labels), dtype=int)

        # Counting entries of Confusion Matrix
        for true_label, pred_label in zip(y, y_pred):
            confusion_matrix[int(true_label), int(pred_label)] += 1
            
        # Computing accuracy
        accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)
        
        return accuracy, confusion_matrix
    
    

In [388]:
temp = PM_HIGH_PREDICTOR(K = 2)

In [360]:
np.max(beijing_np[:, 10])

np.float64(1.0)

In [389]:
temp.fit(beijing_np[:, :10], verbose=True)

ITERATION 1. DISTANCE: 3.3096248856360773
ITERATION 2. DISTANCE: 1.1968893384840977
ITERATION 3. DISTANCE: 0.5763703421288812
ITERATION 4. DISTANCE: 0.1985946900972722
ITERATION 5. DISTANCE: 0.0874702164151685
ITERATION 6. DISTANCE: 0.03119307557705164
ITERATION 7. DISTANCE: 0.01824931274281722
ITERATION 8. DISTANCE: 0.009900144767565244
ITERATION 9. DISTANCE: 0.003912845591415147
ITERATION 10. DISTANCE: 0.0


In [384]:
len(beijing_np[0].shape)

1

In [390]:
temp.predict(beijing_np[:, :10])

array([0, 0, 0, ..., 0, 0, 0], shape=(2071,))

In [392]:
temp.score(beijing_np[:, :10], beijing_np[:, 10])

(np.float64(0.5519072911636891),
 array([[707, 715],
        [213, 436]]))